In [1]:
import os

In [2]:
os.chdir('../')
%pwd

'/home/paladin/Downloads/Bixi-OD-Matrix-Prediction/Bixi-OD-Matrix-Prediction'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class PrepareCNNAutoencoderBaseModelConfig:
    root_dir: Path    
    base_od_model_path: Path
    base_tensor_model_path: Path      
    params_od_size: list
    params_tensor_size: list
    params_learning_rate: float

In [4]:
from timeseriesPredictor.constants import *
from timeseriesPredictor.utils import read_yaml, create_directories

In [5]:
class configurationManeger:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 secret_filepath = SECRET_FILE_PATH,                 
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath) 
        self.secret = read_yaml(secret_filepath)        
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_prepare_autoencoder_base_model_config(self) -> PrepareCNNAutoencoderBaseModelConfig:
        config = self.config.prepare_autoencoder_base_model
        
        create_directories([config.root_dir])

        prepare_autoencoder_base_model_config = PrepareCNNAutoencoderBaseModelConfig(
            root_dir = config.root_dir,           
            base_od_model_path = config.base_od_model_path,    
            base_tensor_model_path = config.base_tensor_model_path,                   
            params_od_size = self.params.OD_SIZE,
            params_tensor_size= self.params.TENSOR_SIZE,
            params_learning_rate = self.params.LEARNING_RATE,          

        )

        return prepare_autoencoder_base_model_config

In [6]:
from timeseriesPredictor.logger import logging
import keras

2023-08-17 09:50:14.041472: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-17 09:50:14.109819: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-17 09:50:14.110933: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-17 09:50:15.282990: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
class PrepareCNNAutoencoderBaseModel:
    def __init__(self, config: PrepareCNNAutoencoderBaseModelConfig):
        self.config = config

    @staticmethod
    def save_model(path:Path, model:keras.Model):
        model.save(path)

    @staticmethod
    def _prepare_full_model(learning_rate, input_shape, tensor=False):

        if tensor:
            upsampling1 = (4,4)
            upsampling2 = (2,2)            
        else:
            upsampling1 = (4,4)
            upsampling2 = (2,2)
        
        inputs = keras.layers.Input(shape= input_shape, name='matrix_array') 
        # encoder
        encoded = keras.layers.Convolution2D(filters=18, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
        encoded = keras.layers.Convolution2D(filters=3, kernel_size=(3, 3), activation='relu', padding='same')(encoded)        
        encoded = keras.layers.BatchNormalization()(encoded)        
        encoded = keras.layers.MaxPooling2D(pool_size=(2, 2))(encoded)
        encoded = keras.layers.Dropout(0.25)(encoded)     
        bottleneck = keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same', name='encoder')(encoded)        
        
        # decoder
        decoded = keras.layers.UpSampling2D(upsampling1)(bottleneck)        
        decoded = keras.layers.Convolution2D(filters=18, kernel_size=(3, 3), activation='relu', padding='same')(decoded)
        decoded = keras.layers.Convolution2D(filters=18, kernel_size=(3, 3), activation='relu', padding='same')(decoded)
        decoded = keras.layers.BatchNormalization()(decoded)             
        decoded = keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same')(decoded)
        decoded = keras.layers.Dropout(0.25)(decoded)
        decoded = keras.layers.UpSampling2D(upsampling2)(decoded)
        decoded = keras.layers.Convolution2D(filters=1, kernel_size=(3, 3), activation='linear', padding='same', name='decoder')(decoded)   
           
        autoencoder = keras.Model(inputs=inputs, outputs=decoded)           

        autoencoder.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss = 'mean_squared_error' ,
            metrics=['MSE']           
                           )
        
        autoencoder.summary()
        return autoencoder
    
    def update_model(self):
        input_shapes = [self.config.params_od_size, self.config.params_tensor_size]
        model_pathes = [self.config.base_od_model_path, self.config.base_tensor_model_path]
        tensors = [False, True]
        for input_shape, model_path, tensor  in zip(input_shapes, model_pathes, tensors):
            self.full_model = self._prepare_full_model(        
            learning_rate = self.config.params_learning_rate,
            input_shape = input_shape,
            tensor = tensor
            )

            self.save_model(path=model_path , model=self.full_model)
            logging.info(f"Autoencoder base model saved at {model_path}!")

    

In [8]:
import sys
from timeseriesPredictor.exception import CustomException

In [9]:
try:
    config = configurationManeger()
    prepare_auto_base_model_config = config.get_prepare_autoencoder_base_model_config()
    prepare_auto_base_model = PrepareCNNAutoencoderBaseModel(config=prepare_auto_base_model_config)    
    prepare_auto_base_model.update_model()

except Exception as e:
    raise CustomException(e, sys)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 matrix_array (InputLayer)   [(None, 64, 64, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 18)        180       
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 3)         489       
                                                                 
 batch_normalization (Batch  (None, 64, 64, 3)         12        
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 32, 32, 3)         0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 32, 32, 3)         0     

/home/paladin/Downloads/Bixi-OD-Matrix-Prediction/Bixi-OD-Matrix-Prediction/venv/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


                                                                 
 conv2d_4 (Conv2D)           (None, 100, 64, 18)       180       
                                                                 
 conv2d_5 (Conv2D)           (None, 100, 64, 3)        489       
                                                                 
 batch_normalization_2 (Bat  (None, 100, 64, 3)        12        
 chNormalization)                                                
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 50, 32, 3)         0         
 g2D)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 50, 32, 3)         0         
                                                                 
 encoder (MaxPooling2D)      (None, 25, 16, 3)         0         
                                                                 
 up_sampli